In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [1]:

#This function takes articles dataframe, it will aggregate it to get monthly views for each article
# apply segmented regression on each article and test if the article shows a chilling effect 
def verify_chill (articles):
    result= pd.DataFrame(articles["article"])
    result["label"]=0
    result=result.set_index("article")
    data= articles.groupby(["article", pd.Grouper(key='date', freq='M')]).sum()
    data= data.reset_index()
    data= data.drop(columns=["Unnamed: 0"])
    article_names= articles['article'][0:3]
    print(article_names)
    june_2013 = pd.to_datetime('2013-06-30')
    june_2013_index = 18
    
    for article in article_names:
        #preparing the dataframe of each article before computing the regression
        pageviews= data[data.article==article]
        pageviews['intervention']= (pageviews.date >= june_2013).astype(int)
        pageviews['time'] = range(1,len(pageviews) + 1)
        pageviews['post_slope'] = (pageviews['time'] - june_2013_index + 1) * \
                                         pageviews['intervention']
        #regression model
        model = smf.ols(formula='views ~ time + C(intervention) + post_slope ', data=pageviews)
        res = model.fit()
        pvalue_intervention= res.pvalues[1]
        pvalue_postslope= res.pvalues[2]
        
        #maybe we will change the condition to consider if an article has a chilling effect
        if pvalue_intervention < 0.05 or pvalue_postslope < 0.05 :
            result[result.index==article]=1
            
            
    return result
        
    